In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style="darkgrid")
from time import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
# Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA    

# Pretty display for notebooks
%matplotlib inline
from random import shuffle

In [ ]:
# without accuracy
import time
from sklearn.model_selection import train_test_split
# import pulp as p 
# from random import *
data= pd.read_csv('data/propublica/compass.csv', skipinitialspace=True)
# data = data1[data1["race"].isin(["african-american", "caucasian"])]

print(data['African_American'].value_counts())
print(data['Female'].value_counts())
# print(data.shape[0],data.shape[1])
data=data.drop(columns=['id'])
# print(data.head())
# Age_Above_FourtyFive,Age_Below_TwentyFive, African_American,Female,  Two_yr_Recidivism  



data_c = data.drop(columns=[ 'Two_yr_Recidivism' ])
# print(sens)
print(data_c.head())
r=data[['Two_yr_Recidivism']]

X_train, X_test, Y_train, Y_test = train_test_split(data_c , r, test_size = 0.3, random_state=0, shuffle=True)

X_test.reset_index(drop=True, inplace=True)
# Y_test_pred.reset_index()

Y_test.reset_index(drop=True, inplace=True)

print(X_test)
#print(Y_test_pred)
print(Y_test)
sens=X_test[['African_American','Female']]
print(sens)
p=sens.shape[0]


sens1 = pd.get_dummies(sens, columns=['African_American','Female'], prefix =['african_american','female'])
sensitive = sens1.T
print(sensitive)



1    3175
0    2103
Name: African_American, dtype: int64
0    4247
1    1031
Name: Female, dtype: int64
   Number_of_Priors  score_factor  ...  Female  Misdemeanor
0                 0             0  ...       0            0
1                 4             0  ...       0            0
2                14             1  ...       0            0
3                 0             0  ...       1            1
4                 0             0  ...       0            0

[5 rows x 7 columns]
      Number_of_Priors  score_factor  ...  Female  Misdemeanor
0                    0             1  ...       0            1
1                    2             0  ...       0            1
2                    5             0  ...       0            0
3                    2             1  ...       0            0
4                    1             0  ...       0            0
...                ...           ...  ...     ...          ...
1579                25             1  ...       0            1
1580      

In [ ]:
y_train = Y_train
y_test = Y_test          

In [ ]:
from sklearn.model_selection import cross_val_score  
from sklearn.svm import SVC


rf = RandomForestClassifier(n_estimators=251, max_depth=None, min_samples_split=50, random_state=0)

#svm = SVC(kernel='rbf', random_state=0, gamma=.1, C=10.0,probability=True)
rf.fit(X_train, y_train)
print('The accuracy of the Random_Forest classifier on training data is {:.2f}'.format(rf.score(X_train, y_train)))
print('The accuracy of the Random_Forest classifier on test data is {:.2f}'.format(rf.score(X_test, y_test)))
print('####Train prediction Label###############################################')
y_train_pred=rf.predict(X_train)
#print(y_1)
y_test_pred=rf.predict(X_test)
e=rf.predict_proba(X_test)
print(e)
print(y_test_pred)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


The accuracy of the Random_Forest classifier on training data is 0.71
The accuracy of the Random_Forest classifier on test data is 0.68
####Train prediction Label###############################################
[[0.43261605 0.56738395]
 [0.74575215 0.25424785]
 [0.59671785 0.40328215]
 ...
 [0.22524897 0.77475103]
 [0.42631319 0.57368681]
 [0.56690856 0.43309144]]
[1 0 0 ... 1 1 0]


In [ ]:
sens=X_test[['African_American', 'Female']]
sens_train_race= X_train[['African_American']]
sens_train = X_train[['African_American', 'Female']]
sens_train_sex = X_train[['Female']]

sens_test_race = X_test[['African_American']]
sens_test_sex = X_test[['Female']]

print(sens)
sens1=pd.get_dummies(sens, columns=['African_American','Female'], prefix =['African_American','Female'])
sensitive=sens1.T
print(sensitive) 

      African_American  Female
0                    1       0
1                    1       0
2                    0       0
3                    1       0
4                    1       0
...                ...     ...
1579                 1       0
1580                 1       1
1581                 1       0
1582                 1       0
1583                 1       1

[1584 rows x 2 columns]
                    0     1     2     3     4     ...  1579  1580  1581  1582  1583
African_American_0     0     0     1     0     0  ...     0     0     0     0     0
African_American_1     1     1     0     1     1  ...     1     1     1     1     1
Female_0               1     1     1     1     1  ...     1     0     1     1     0
Female_1               0     0     0     0     0  ...     0     1     0     0     1

[4 rows x 1584 columns]


In [ ]:
#----------------------------------FPR, TPR computation--------------------------------------------
print(y_test.iloc[10])
from random import *

def find_eo_stats_multiple(y,y_pred):
    m = 4
    sens_stats = np.zeros((4,m), dtype = int)
    sens_acc = np.zeros(m, dtype = float)
    sizes = np.zeros(m, dtype = int)
    #first row positives, second row negatives, third row true positive, fourth row false positive
    
    
    for i in range(m):
        for j in range(len(y)):
            if(sensitive.iloc[i,j] == 1):
                sizes[i] = sizes[i]+1
                if(y_pred[j]==1):
                    sens_acc[i] = sens_acc[i] + 1
                if(y.iloc[j,0]==1):
                    sens_stats[0][i] =sens_stats[0][i] + 1
                if(y.iloc[j,0]==1 and y_pred[j]==1):
                    sens_stats[2][i] =sens_stats[2][i] + 1
                if(y.iloc[j,0]==0):
                    sens_stats[1][i] =sens_stats[1][i] + 1
                if(y.iloc[j,0]==0 and y_pred[j]==1):
                    sens_stats[3][i] =sens_stats[3][i] + 1    
        sens_acc[i] = sens_acc[i]/sizes[i]
        #print(sens_acc[i],sizes[i])
        
    
    TPR = np.zeros(m,dtype=float)
    FPR = np.zeros(m,dtype=float)
    
    accu = 0
    n = len(y)
    for i in range(n):
        if(y.iloc[i,0] == y_pred[i]):
            accu = accu+1
    
    accu = accu/n
    
    max_tpr = -1 
    min_tpr = 2
    max_fpr = -1
    min_fpr = 2
    
    for i in range(m):
        TPR[i] = sens_stats[2][i]/sens_stats[0][i]
        FPR[i] = sens_stats[3][i]/sens_stats[1][i]
        if(TPR[i] >= max_tpr):
            max_tpr = TPR[i]
        if(TPR[i] <= min_tpr):
            min_tpr = TPR[i]
        if(FPR[i] >= max_fpr):
            max_fpr = FPR[i]
        if(FPR[i] <= min_fpr):
            min_fpr = FPR[i]
    
    for i in  range(m):
        print("TPR of the ",i,"th sensitive groups is",TPR[i])
        
    print("=====================")    
    for i in  range(m):
        print("FPR of the ",i,"th sensitive groups is",FPR[i])
    
    print("=====================")
    for i in  range(m):    
        print("Acceptance Rate of the ",i,"th sensitive groups is",sens_acc[i])
    
    DEO = abs(max_tpr-min_tpr) + abs(max_fpr-min_fpr)
    
    print("The Difference of Equalized odds of the classifier is=",DEO)
    print("Accuracy of the classifier",accu)
    
#find_eo_stats_multiple(y_test,y_pred_CEO)
#find_eo_stats_multiple(y_test,y_test_pred)



        

Two_yr_Recidivism    0
Name: 10, dtype: int64


In [ ]:
pip install pulp

     |████████████████████████████████| 14.2 MB 4.7 MB/s 


In [ ]:
# bilal - am_ind asian black other white female male(actual precision sequence)
# 6         5          4          2                1              0                  3

# ours -s_male, s_female  r_white, r_black, r_asian-pac-islander','r_amer-indian-eskimo','r_other
           
# beta=[6         5          4          2                1              0                  3]

# beta=[beta[6], beta[5],beta[4],beta[2],beta[1],beta[0],beta[3]]


#bilal -female male  am_ind asian  black other white (actual acceptance rate sequence)
#          0      1     2       3     4      5    6   
# ours -s_male, s_female  r_white, r_black, r_asian-pac-islander','r_amer-indian-eskimo', 'r_other
           
# beta=[1       0        6          4            3                   2                    5]


#NG
import time
#import gurobipy as gp
#from gurobipy import GRB
import pulp as p 
def lp_equalized_odds(data1,eps,y_test_pred,e,beta_avg,alpha):
    import pulp as p 
    import math
    
    
    m=data1.shape[0]
    n=data1.shape[1]
    print('dimension of data')
    print(m,n)
    
        
    ############### #  SORTED for ACCURACY ONLY ####
    m=4
    h1=[]
    key1=[]
    cost=np.zeros(n,dtype=int)
    data2=np.zeros((m,n),dtype=int)
    
    for i in range(n):
            h1.append(e[i][1])
            key1.append(i)

        
#print(hc)
#     print(key1)
    
    for i in range(1,len(h1)):
        for j in range(i,0,-1):
            var=0
            var2=0
            if h1[j-1]<h1[j]:
                index=j
                var=h1[j]
                h1[j]=h1[j-1]
                h1[j-1]=var

                var2=key1[j]
                key1[j]=key1[j-1]
                key1[j-1]=var2
            else:
                break
    

    
    
    for j in range(len(key1)):    
         data2[0][key1[j]]=j+1
    
    for j in range(n):
        summ=0
        summ=summ+data2[0][j] 
        cost[j]=summ

    Lp_prob = p.LpProblem('Problem', p.LpMinimize)  
    solver = p.getSolver('PULP_CBC_CMD', timeLimit=20)
   
    
#     X=np.zeros(n+1,dtype=p.LpVariable)
    X=np.zeros(n+m+1,dtype=p.LpVariable)
    Y=np.zeros(m,dtype=p.LpVariable)
    
    sizes=np.zeros(m,dtype=int)
#     report_index(index,data1,e):  
    max_size=0
    for i in range(m):
        count=0
        for j in range(n):
            if data1[i][j]==1:
                count=count+1 
        if count>max_size:
            max_size=count
        sizes[i]=count
    print(sizes)    
    ############################
    
    
    
    
    ###############################
    beta_actual = [0.2708333333333333, 0.5604166666666667, 0.4634920634920635, 0.37962962962962965]



    
    
    select_sizes=np.zeros(m,dtype=int)
   
    size_final=np.zeros(m,dtype=int)

    for i in range(m):
        var1 = str(n+100+i)
        Y[i]=p.LpVariable(var1,lowBound=0,upBound=1,cat='Continuous')
    
    for i in range(n):
        var1=str(i)       
        X[i]=p.LpVariable(var1,lowBound=0,upBound=1,cat='Integer')
   
    X[n]=p.LpVariable(str(n),lowBound=0,upBound=1,cat='Continuous')  

    tpr = p.LpVariable(str(n+200),lowBound=0,upBound=1,cat='Continuous')  
    fpr = p.LpVariable(str(n+201),lowBound=0,upBound=1,cat='Continuous')  

#     for i in range(m):
#         k=n+i+1
#         var1=str(k)     
#         alpha=(((sizes[i])*(sizes[i]+1))/2)
#         X[i]=p.LpVariable(var1,lowBound=(((beta*sizes[i])*(beta*sizes[i]+1))/2),upBound=alpha,cat='Continuous')
    
        
#     X[n]=  p.LpVariable("z1",lowBound=0)
    #X[n+1]=  p.LpVariable("z2",lowBound=0)
  

    #########objective function#####################
    
#     Lp_prob += 2*X[n+1]+10*X[n+2]+9*X[n+3]+3*X[n+4]
    #alpha=0.8
    #beta_avg = 0.10
    Lp_prob+= p.lpSum([(X[j])*cost[j] for j in range(n)]) 
    #Lp_prob+=1  
    
    #Lp_prob += Y[0]*sizes[0] + Y[1]*sizes[1] >= p.lpSum([Y[j]*sizes[j] for j in np.arange(2,6)])
    #Lp_prob += Y[0]*sizes[0] + Y[1]*sizes[1] <= p.lpSum([Y[j]*sizes[j] for j in np.arange(2,6)])
    
    ##############constraint#################
    #first select the  the number of make female in test data
    #then apply the equalized odd constraints assuming 
    #look at all males which have been predicted positve/and all the females predicted negative
    F_test = 0
    M_test = 0
        
    #for i in range(len(y_test)):
    #    if(data1[0][i]==1 and y_test.iloc[i]==1):
    #        M_test= M_test+1
    #    elif(data1[1][i]==1 and y_test.iloc[i]==1):
    #        F_test= F_test+1
    test_count = np.zeros(m, dtype = int)

    for i in range(len(y_test)):
      for j in range(m): 
        if(data1[j][i]==1 and y_test_pred[i]==1):
            test_count[j] = test_count[j] +1
                
    
    #Lp_prob += (p.lpSum([(X[j])*(data1[0][j])*y_test_pred[j] for j in range(n) if y_test_pred[j]==1])/M_test) <= (p.lpSum([(X[j])*(data1[1][j])*y_test_pred[j] for j in range(n) if y_test_pred[j]==1])/F_test) + 0.0009
    #Lp_prob += (p.lpSum([(X[j])*(data1[0][j])*(1-y_test_pred[j]) for j in range(n) if y_test_pred[j]==0])/(sizes[0]-M_test)) <= (p.lpSum([(X[j])*(data1[1][j])*(1-y_test_pred[j]) for j in range(n) if y_test_pred[j]==0])/(sizes[1]-F_test))+ 0.0009
    for i in range(m):   #TPR constraints
      Lp_prob += (1/test_count[i])*p.lpSum([(X[j])*(data1[i][j])*y_test_pred[j] for j in range(n) if (y_test_pred[j]==1) ]) >= tpr 
      Lp_prob += (1/test_count[i])*p.lpSum([(X[j])*(data1[i][j])*y_test_pred[j] for j in range(n) if (y_test_pred[j]==1 )]) <= tpr + 0.1
    for i in range(m):    #FPR constraints
      Lp_prob += (1/(sizes[i]-test_count[i]))*p.lpSum([(X[j])*(data1[i][j])*(1-y_test_pred[j]) for j in range(n) if (y_test_pred[j]==0)]) >= fpr
      Lp_prob += (1/(sizes[i]-test_count[i]))*p.lpSum([(X[j])*(data1[i][j])*(1-y_test_pred[j]) for j in range(n) if (y_test_pred[j]==0)]) <= fpr + 0.1

    #Lp_prob += F_test*p.lpSum([(X[j])*(data1[0][j])*y_test_pred[j] for j in range(n) if (y_test_pred[j]==1) ]) <= M_test*p.lpSum([(X[j])*(data1[1][j])*y_test_pred[j] for j in range(n) if (y_test_pred[j]==1 )]) + 0.004
    #Lp_prob += (sizes[1]-F_test)*p.lpSum([(X[j])*(data1[0][j])*(1-y_test_pred[j]) for j in range(n) if (y_test_pred[j]==0)]) <= (sizes[0]-M_test)*p.lpSum([(X[j])*(data1[1][j])*(1-y_test_pred[j]) for j in range(n) if (y_test_pred[j]==0 )]) + 0.004
    


    #Lp_prob += F_test*p.lpSum([(X[j])*(data1[0][j])*y_test_pred[j] for j in range(n) if (y_test_pred[j]==1 and y_test.iloc[j]==1) ]) <= M_test*p.lpSum([(X[j])*(data1[1][j])*y_test_pred[j] for j in range(n) if (y_test_pred[j]==1 and y_test.iloc[j]==1)]) + 0.01
    #Lp_prob += (sizes[1]-F_test)*p.lpSum([(X[j])*(data1[0][j])*(1-y_test_pred[j]) for j in range(n) if (y_test_pred[j]==0 and y_test.iloc[j]==0)]) <= (sizes[0]-M_test)*p.lpSum([(X[j])*(data1[1][j])*(1-y_test_pred[j]) for j in range(n) if (y_test_pred[j]==0 and y_test.iloc[j]==0)]) + 0.01
    
    #Lp_prob += p.lpSum([(X[j])*(data1[0][j])*y_test.iloc[j] for j in range(n) if y_test.iloc[j]==1])/M_test <= p.lpSum([(X[j])*(data1[1][j])*y_test.iloc[j] for j in range(n) if y_test.iloc[j]==1])/F_test + 0.004
    #Lp_prob += (sizes[1]-F_test)*p.lpSum([(X[j])*(data1[0][j])*(1-y_test.iloc[j]) for j in range(n) if y_test.iloc[j]==0])/M_test <= (sizes[0]-M_test)*p.lpSum([(X[j])*(data1[1][j])*(1-y_test.iloc[j]) for j in range(n) if y_test.iloc[j]==0])/F_test + 0.004
    
    for i in range(m):
      #  if i<m:
            Lp_prob += p.lpSum([(X[j])*(data1[i][j]) for j in range(n)]) >= Y[i]*sizes[i]
            Lp_prob += p.lpSum([(X[j])*(data1[i][j]) for j in range(n)]) <= (Y[i]+0.1)*sizes[i]
    
    for i in range(m):
        if beta_actual[i] >= beta_avg:
            Lp_prob += Y[i] >= (1-alpha)*beta_actual[i] + alpha*beta_avg
            Lp_prob += Y[i] <= beta_actual[i]
        else:
            Lp_prob += Y[i] >= (1-alpha)*beta_actual[i] + alpha*beta_avg
            Lp_prob += Y[i] <= beta_avg 
    
           
    #Lp_prob+= p.lpSum([(X[j])*cost[j] for j in range(n)])>=100
        
    #####################################
    #solver = p.CPLEX_PY()
    #solver.buildSolverModel(Lp_prob)
    #Lp_prob.solverModel.parameters.timelimit.set(60)
    #solver.callSolver(P)
    #status = solver.findSolutionValues(Lp_prob)
    #################################################################
    status = Lp_prob.solve(solver)   # Solver 
    print(p.LpStatus[status]) 
    print("objective is:")        
    print(p.value(Lp_prob.objective))
    print("discripency is:") 
    print(p.value(X[n]))
    x=np.zeros(n,dtype=float)

   # The solution status 
    Synth1={}
    Synth2={}
    # # Printing the final solution 
    for i in range(n):
        if(p.value(X[i])==1):
            Synth1[i]=1 
            Synth2[i]=-1
#             if(data1[2][i]==1):
#                 print("no")
        else:
            Synth1[i]=-1
            Synth2[i]=1
    Synthu1=Synth1  
    Synthu2=Synth2  
    
              
    return Synthu1,Synthu2   


In [ ]:
pip install pulp

     |████████████████████████████████| 41.2 MB 77 kB/s 


In [ ]:
#without accuracy ---> 2
def main2(datax, y_test, y_test_pred,e): 
        
    n=datax.shape[1]
    s=datax.shape[0]    
    data = np.zeros((s, n), dtype = int)
    
    r = np.zeros(n, dtype = int) 
    
    for i in range(n):
        if int(y_test.iloc[i])==1 :
            r[i]=1
        else :
            r[i]= -1  
    
    r2 = np.zeros(n, dtype = int) 
    for i in range(n):
        if int(y_test_pred[i])==1 :
            r2[i]=1
        else :
            r2[i]= -1          
    ar=[]
    
    for j in range(s):
        print("sensitive attribute ",(j+1)) 
        a=0
        b=0
        acc1=0
        acc2=0
        for i in range(n):
                data[j][i]= datax.iloc[j,i]
                if data[j][i]== 1 :
                    a=a+1
                    if r[i]==1:
                         acc1=acc1+1 

        print("ACTUAL----------total ,accepted, aceeptance rate:")             
        a1=float(acc1/a)
        print(a)
        
        print(acc1)
        print(a1)
        ar.append(a1)
        
    maxi= max(ar)
    mini= min(ar)
    DP=float(maxi-mini)
    print("data acceptance rates")
    print(ar)
    print("data DP")
    print(DP)
    
    ar=[]
    
    for j in range(s):
        print("sensitive attribute ",(j+1)) 
        a=0
        b=0
        acc1=0
        acc2=0
        prec=0
        reca=0
        accur=0
        FP=0
        FN=0
        TP=0
        TN=0
        for i in range(n):
             if data[j][i]== 1 :
                    a=a+1
                    if r2[i]==1:
                        acc1=acc1+1 
                        if r[i]==1:
                            TP=TP+1
                        else:
                             FP=FP+1                
                    else:
                        if r[i]==1:
                            FN=FN+1
                        else:
                            TN=TN+1    
        
        print("prec reca accuracy for each sens") 
        prec= float(TP/(TP+FP))
        reca= float(TP/(TP+FN))
        accur= float((TP+TN)/a)
        print(prec,reca,accur)
        
        print("RanomForest----------total , accepted, aceeptance rate:")             
        
        a1=float(acc1/a)
        print(a)
        
        print(acc1)
        print(a1)
        ar.append(a1)
        
    maxi= max(ar)
    mini= min(ar)
    DP=float(maxi-mini)
    print("data acceptance rates")
    print(ar)
    print("data DP")
    print(DP) 
    
    print("SVM accuracy--------------------------")
    prec=0
    reca=0
    accur=0
    FP=0
    FN=0
    TP=0
    TN=0
    for i in range(n):
            if r2[i]==1:
                acc1=acc1+1 
                if r[i]==1:
                    TP=TP+1
                else:
                     FP=FP+1                
            else:
                if r[i]==1:
                     FN=FN+1
                else:
                     TN=TN+1    

        
    prec= float(TP/(TP+FP))
    reca= float(TP/(TP+FN))
    accur= float((TP+TN)/n)
    print(prec,reca,accur)
    
    
#     delta1=[.70,.75,.80,.85,.90,.95]
    #gamma=.05,.06,.07
    #delta1=[.80,.85,.90,.95]
# (for reproducibility)  

# delta1=[.8], gama=[.1], epsilon=[.05]  
# delta1=[.8], gama=[.15], epsilon=[.01]
 
#     delta1=np.arange(1,.79,-.01)
    delta=1
#     gama=[.05,.1,.15,.2,.25]
#     epsilon=[.01,.02,.05,.1,.15,.20,.25,.30,.35,.40,.50]

#ADULT ZAFAR =? epsilon=[0.088 ,0.1656, 0.168,  0.211, 0.251 ] 
 
#agarwal=> epsilon=[ 0.071, 0.1271, 0.2437, 0.27 ]
 

    #gama=[0.0869, 0.0521,0.0782, 0.0608,0.0434, 0.1,0.069,0.0434,0.034]
    epsilon=[.1]
    beta_converge = [0.15,0.20]
    alpha = [0]
    
    zero_one = np.zeros(n, dtype = int) 
    
    fi= np.zeros(n,dtype=int) 
#     for delta in delta1:
    #4 gamma=[0.175442,    0.142103, 0.166039,    0.164754,  0.153465,    0.14,  0.104348   ]
    #lp_equalized_odds_no_beta(data1,eps,y_test,e,beta_avg,alpha)
    #1 gamma=[0.259147,   0.0730028, 0.210139, 0.0893443, 0.306931, 0.0933333,  0.0347826]
    #gamma=[0.196178,0.126722,   0.179654, 0.140164,     0.153465,   0.133333,  0.0695652]

  
    gamma = [0.175442,    0.142103, 0.166039,    0.164754,  0.153465,    0.14,  0.104348 ]
    for eps in epsilon:
        for beta_avg in beta_converge:
            print("----------------This is for covergence at beta = ",beta_avg, " ----------------")
            for a in alpha:
                u1,u2=lp_equalized_odds(data,eps,y_test_pred,e,beta_avg,a)
                #######################Disp_impact#######################  
                print("gamma-epsilon-delta",gamma,eps,delta)
                accu_all=[]
                DP_all=[]
                precision_all=[]
                recall_all=[]
                acceptance_rate=np.zeros((7,28),dtype=float)
                count=0
                print("<--------------------------------------->")
            #        print("iteration t",t)
            #                 for alpha in np.arange(0,1.05,0.05):
            #                     print("alpha: ",alpha)
            #                     for i in range(n):

            #                         z=random()
            #                         if z < alpha:
            #                                fi[i]= u1[i] 

            #                         else:
            #                                fi[i]= r2[i]
                
                for i in range(n):
                    fi[i] = u1[i]
                    if (fi[i]==1):
                        zero_one[i] = 1
                    else:
                        zero_one[i] = 0
                ar=[]
                #find_eo_stats(y_test,zero_one)
                find_eo_stats_multiple(y_test,zero_one)


                for j in range(s):
                    print("sensitive attribute ",(j+1)) 

                    TP=0
                    FP=0
                    FN=0
                    TN=0
                    precision=0
                    recall=0
                    for i in range(n):
                         if data[j][i]== 1 :                        
                            if fi[i]==1 and r[i]==1:
                                TP=TP+1
                            if fi[i]==1 and r[i]==-1:
                                FP=FP+1 
                            if fi[i]==-1 and r[i]==1:
                                FN=FN+1
                            if fi[i]==-1 and r[i]==-1:
                                TN=TN+1    
                    if TP+FP !=0:
                        precision=float(TP/(TP+FP))
                    print("precision",precision)
                    if TP+FN !=0:    
                        recall=float(TP/(TP+FN))
                    print("recall",recall)
                    if FP+TN !=0:
                        fpr = float(FP/(FP+TN))
                    print("FPR", fpr)    
                    print("TP,FP,TN,FN")
                    print(TP,FP,TN,FN)
                    a=0
                    b=0
                    acc1=0
                    acc2=0
                    for i in range(n):
                            if data[j][i]== 1 :
                                a=a+1
                                if fi[i]==1:
                                     acc1=acc1+1 

            #                         print("total ,fair accepted, aceeptance rate:")             
                    a1=float(acc1/a)



            #                         print(a)
            #                         print(acc1)
            #                         print(a1)
                    ar.append(a1)

                count = count+1
                maxi=max(ar)
                mini= min(ar)
                DP=float(maxi-mini)
                print("acceptance rates")
                print(ar)
                print("DP")
                print(DP)
                f_acc=0
                for i in range(n):
                     if fi[i] == r[i]:
                            f_acc=f_acc+1
                f_acc_l=float((f_acc*100)/n) 

#######################################################################33   

#                         print("sensitive attribute ",(j+1)) 





        TP=0
        FP=0
        FN=0
        TN=0
        precision=0
        recall=0
        for i in range(n):
                if fi[i]==1 and r[i]==1:
                    TP=TP+1
                if fi[i]==1 and r[i]==-1:
                    FP=FP+1 
                if fi[i]==-1 and r[i]==1:
                    FN=FN+1
                if fi[i]==-1 and r[i]==-1:
                    TN=TN+1    

        if TP+FP!=0:
            precision=float(TP/(TP+FP))
        print("precision",precision)
        if TP+FN!=0:
            recall=float(TP/(TP+FN))    

        print("recall",recall)
        
        accu = float((TP + TN)/n)
        print("TP,FP,TN,FN")
        print(TP,FP,TN,FN)
#       print("total ,fair accepted, aceeptance rate:")             
        a1=float(acc1/a)


    print("<--------------------------------------->")
    alpha_weight=np.arange(0,1.05,.05)        
    return accu_all,DP_all,acceptance_rate,alpha_weight

In [ ]:
accu_all,DP_all,acceptance_rate,alpha_weight = main2(sensitive, y_test, y_test_pred,e)

sensitive attribute  1
ACTUAL----------total ,accepted, aceeptance rate:
624
238
0.3814102564102564
sensitive attribute  2
ACTUAL----------total ,accepted, aceeptance rate:
960
484
0.5041666666666667
sensitive attribute  3
ACTUAL----------total ,accepted, aceeptance rate:
1260
599
0.4753968253968254
sensitive attribute  4
ACTUAL----------total ,accepted, aceeptance rate:
324
123
0.37962962962962965
data acceptance rates
[0.3814102564102564, 0.5041666666666667, 0.4753968253968254, 0.37962962962962965]
data DP
0.124537037037037
sensitive attribute  1
prec reca accuracy for each sens
0.6341463414634146 0.4369747899159664 0.6891025641025641
RanomForest----------total , accepted, aceeptance rate:
624
164
0.26282051282051283
sensitive attribute  2
prec reca accuracy for each sens
0.6469534050179212 0.7458677685950413 0.6666666666666666
RanomForest----------total , accepted, aceeptance rate:
960
558
0.58125
sensitive attribute  3
prec reca accuracy for each sens
0.6355283307810107 0.692821368